In [2]:
from sklearn.tree import DecisionTreeClassifier
# 예시 데이터로 아이리스 데이터
# from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#################################################
# graphiz로 트리 모형 시각화
#################################################

# 1. DecisionTree Classifier 생성
dt_clf = DecisionTreeClassifier(random_state=156)

# 2. 예시 데이터: 트레인 셋과 테스트 셋으로 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, 
                                                      iris_data.target, 
                                                      test_size=.2, 
                                                      random_sate=11)
# 3. DecisionTree Classifier 학습
dt_clf.fit(X_traink, y_train)

# 4. graphviz로 시각화
from sklearn.tree import export_graphviz
# export_graphviz()의 호출 결과로 out_file로 지정된 tree.dot 파일을 생성함.
export_graphviz(dt_clf, out_file='tree.dot', class_names=iris_data.target_names, \
                feature_names=iris_data.feature_names, impurity=True, filled=True)

import graphviz
# 위에서 생성된 tree.dot 파일을 Graphviz로 읽어서 jupyter notebook상에서 시각화
with open('tree.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph) # Source에 넣기

In [ ]:
#################################################
# feature_importances_시각화
#################################################

import seaborn as sns
sns.barplot(
    x=iris_data.feature_names,
    y=dt_clf.feature_importances_ # 트리 계의 모델에서는 대부분 feature_importances_가 있음.
)

for name, val in zip(iris_data.feature_names, dt_clf.feature_importances_):
    print ("{0}의 중요도는: {1: .3f}".format(name, val))
    

In [ ]:
#################################################
# 결정 트리 정확도 구하는 순서
#################################################

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)
pred = df_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
# 결정트리 예측 정확도
print(accuracy)

# 결정트리 하이퍼 파라미터 추출
dt_clf.get_params

In [ ]:
#########################################################
# GridSearchCV 사용하여 최고의 하이퍼 파라미터를 찾고 정확도 예측하기#
#########################################################
from sklearn.model_selection import GridSearchCV
params = {
    'max_depth' : [8,10,12,16,20],
    'min_samples_split' : [16,24]
}

grid_cv = GridSearchCV(dt_clf, param_grid = params, scoring='accuracy', cv=5, verbose = 1)
grid_cv.fit(X_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치 : {0: .4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

# 결과를 cv_results_로 구할 수 있음.
cv_result_df = pd.DataFrame(grid_cv.cv_results_)
cv_result_df[['param_max_depth', 'mean_test_score', 'rank_test_score']]

# best_estimator_로 predict
best_dt_clf = grid_cv.best_estimator_
pred1 = best_dt_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도: {0: .4f}'.format(accuracy))

# 마지막으로 중요한 features들 확인하기
ftr_importances_values = best_dt_clf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns)
ftr_top_20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top_20, y=ftr_top_20.index)
plt.show()